# Download and process Forest Extent and Height Change, 2000-2020 data into a single geotiff at native resolution

In [1]:
import os
import sys
import shutil
import tarfile
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [13]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
forest_path          = cs.read_from_config(config_file,'forest_path')
forest_2000_url      = cs.read_from_config(config_file,'forest_2000_url')
forest_2020_url      = cs.read_from_config(config_file,'forest_2020_url')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Convert geospatial coordinates to specific download coordinates for this data product

In [4]:
download_coordinates = cs.geospatial_coordinates_to_download_coordinates(download_area, 'glclu2019')

Returning coordinates as type <class 'str'> for use with glclu2019 download code.


### Download data

In [6]:
forest_lon,forest_lat = cs.convert_coordinates_to_glclu2019_download_lists(download_coordinates)

In [7]:
download_folder_2000 = Path(data_path) / geospatial_temp_path / 'forest_height' / 'download' / '2000'
download_folder_2020 = Path(data_path) / geospatial_temp_path / 'forest_height' / 'download' / '2020'

In [8]:
download_folder_2000.mkdir(parents=True, exist_ok=True)
download_folder_2020.mkdir(parents=True, exist_ok=True)

In [14]:
for dl_lon in forest_lon:
    for dl_lat in forest_lat:
        url_2000 = forest_2000_url.format(dl_lat,dl_lon) # Replace placeholder values in url with download coordinates
        cs.download_glclu2019_grid(url_2000,download_folder_2000) # Despite the glcluc name, works for this data too
        
        url_2020 = forest_2020_url.format(dl_lat,dl_lon) 
        cs.download_glclu2019_grid(url_2020,download_folder_2020)

Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_60N_180W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2020/2020_60N_180W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_30N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2020/2020_30N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_60N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2020/2020_60N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_70N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2020/2020_70N_170W.tif
Successfully downloaded https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_80N_170W.tif
Successfully downloaded https://glad.

### Merge

In [15]:
# Define the output file
merged_2000_file = 'forest_height_2000.tif'
merged_2020_file = 'forest_height_2020.tif'
merged_folder = Path(data_path) / geospatial_temp_path / 'forest_height' / 'raw' # We'll end up with two files in here only, so no need to separate into 'map' and 'strata'

In [16]:
cs.merge_glclu2019_files_into_one(merged_2000_file, download_folder_2000, merged_folder, download_area)

In [17]:
cs.merge_glclu2019_files_into_one(merged_2020_file, download_folder_2020, merged_folder, download_area)

### Delete the individual GeoTIFF files to save space

In [19]:
download_folder = Path(data_path) / geospatial_temp_path / 'forest_height' / 'download'
if os.path.isdir(download_folder): 
    shutil.rmtree(download_folder) # main folder that contains the '2000' and '2020' folders